In [28]:
import sys
import seaborn as sns
#!{sys.executable} -m pip install dtw
from dtw import dtw
import librosa, librosa.display, os
import pandas as pd
import numpy as np
from numpy.linalg import norm
from dtw import dtw

path = 'komendy_Kasia/'
signals = list()
dataset_mfcc = list()
labels = open('labels.txt','r').read().split('\n')
set_labels = set(labels)
label_dict = dict()
word_number=dict()
for label in set(labels):
    indices = [i for i, x in enumerate(labels) if x == label]
    label_dict[label] = indices
    word_number[label]=len(indices)
#print(label_dict)
#print(word_number)

In [29]:
feature_dict = dict()
for key in label_dict:
    list_signals = list()
    len_signal = list()
    for el in label_dict[key]:
        filename = "%d.wav"%(el)
        signal, sr = librosa.load(os.path.join(path,filename)) #lista sygnalow
        list_signals.append(signal)
        len_signal.append(len(signal))
    #print(list_signals)
    maks_len = np.max(len_signal)
    mfcc_list = list()
    for signal in list_signals:
        signal = np.pad(signal, (0,maks_len-len(signal)), 'mean')
        mfcc = librosa.feature.mfcc(signal, sr)
        mfcc_list.append(mfcc)
    mean = np.mean( mfcc_list, axis=0 )
    feature_dict[key] = mean
#print(feature_dict)

In [30]:
error = 0

#create result matrix
dataFrame = pd.DataFrame(index = set_labels, columns=set_labels, dtype='float')
dataFrame = dataFrame.fillna(0)

#recognition
for filename in os.listdir('komendy/'):
    y1, sr1 = librosa.load('komendy/'+filename)
    mfcc1 = librosa.feature.mfcc(y1, sr1)
    min_dist = 1000
    for key in feature_dict:
        mfcc2 = feature_dict[key]
        dist, cost, acc_cost, path = dtw(mfcc1.T, mfcc2.T, dist=lambda x, y: norm(x - y, ord=1))
        if dist<min_dist:
            min_dist = dist
            value = key
    index = filename.split('.')[0]
    
    if value == labels[int(index)]:
        print("%s detected correct"%(value))
        dataFrame.loc[labels[int(index)],value]+=1 #count how many times value is recognized as labels[int(index)]
    else:
        print("%s detected incorrect, should be %s"%(value, labels[int(index)]))
        error+=1
        dataFrame.loc[labels[int(index)],value]+=1

print(dataFrame)


RADIO detected incorrect, should be ZAPAL
STOPIEN detected incorrect, should be SWIATLO
WYLACZ detected correct
W detected correct
SYPIALNI detected correct
ZAMKNIJ detected correct
W detected incorrect, should be BRAME
ZWIEKSZ detected correct
LAZIENCE detected incorrect, should be OGRZEWANIE
W detected incorrect, should be O
LAZIENCE detected incorrect, should be JEDEN
STOPIEN detected correct
LAZIENCE detected incorrect, should be ZAKREC
TELEWIZOR detected correct
W detected incorrect, should be WODE
W detected correct
LAZIENCE detected correct
USTAW detected correct
WODE detected incorrect, should be ALARM
PRZYCISZ detected correct
DRZWI detected incorrect, should be RADIO
ZMIEN detected correct
WODE detected incorrect, should be KANAL
WLACZ detected incorrect, should be PODLEJ
TELEWIZOR detected incorrect, should be PODNIES
LAZIENCE detected incorrect, should be KWIATKI
ZAPARZ detected correct
W detected incorrect, should be KAWE
WYLACZ detected correct
GARAZU detected incorrect, 

In [33]:
#macierz procentowa
for i in set_labels:
    for j in set_labels:
        dataFrame.loc[i,j]=dataFrame.loc[i,j]*100/word_number[i]
       
print(dataFrame)
dataFrame.to_excel('results.xls')

            JEDEN  KANAL  KUCHNI  TELEWIZOR  ZAMKNIJ  ZAPAL  SYPIALNI  \
JEDEN        25.0    0.0    0.00        0.0      0.0    0.0       0.0   
KANAL         0.0    0.0    0.00        0.0      0.0    0.0       0.0   
KUCHNI        0.0    0.0   25.00        0.0      0.0    0.0       0.0   
TELEWIZOR     0.0    0.0    0.00      100.0      0.0    0.0       0.0   
ZAMKNIJ       0.0    0.0    0.00        0.0     50.0    0.0       0.0   
ZAPAL         0.0    0.0    0.00        0.0      0.0   25.0       0.0   
SYPIALNI      0.0    0.0    0.00        0.0      0.0    0.0      25.0   
LAZIENCE      0.0    0.0    0.00        0.0      0.0    0.0       0.0   
WLACZ         0.0    0.0    0.00        0.0      0.0    0.0       0.0   
RADIO         0.0    0.0    0.00        0.0      0.0    0.0       0.0   
KAWE          0.0    0.0    0.00        0.0      0.0    0.0       0.0   
OGRZEWANIE    0.0    0.0    0.00        0.0      0.0    0.0       0.0   
ALARM         0.0   25.0    0.00        0.0      0.

In [34]:
cm = sns.light_palette("green", as_cmap=True)

s = dataFrame.style.background_gradient(cmap=cm)
s

,JEDEN,KANAL,KUCHNI,TELEWIZOR,ZAMKNIJ,ZAPAL,SYPIALNI,LAZIENCE,WLACZ,RADIO,KAWE,OGRZEWANIE,ALARM,ZWIEKSZ,STOPIEN,SWIATLO,USTAW,GARAZU,W,ZMIEN,ZAPARZ,ZMYWARKE,ZAGOTUJ,BRAME,PODNIES,O,DO,PRZYCISZ,ROLETY,WYLACZ,DRZWI,WODE,ZAKREC,KWIATKI,OTWORZ,PODLEJ
JEDEN,25,0,0,0,0,0,0,25,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,50,0,0,0,0,0
KANAL,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,25,0,0,0,0,0,0,0,0,0,75,0,0,0,0
KUCHNI,0,0,25,0,0,0,0,0,0,25,0,0,0,0,0,0,0,0,25,0,0,0,0,0,0,0,0,0,0,0,0,25,0,0,0,0
TELEWIZOR,0,0,0,100,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
ZAMKNIJ,0,0,0,0,50,0,0,0,0,25,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,25,0,0,0,0,0
ZAPAL,0,0,0,0,0,25,0,0,0,50,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,25,0,0,0,0,0,0,0,0,0
SYPIALNI,0,0,0,0,0,0,25,25,0,0,0,0,0,0,0,0,0,0,0,25,0,0,0,0,0,0,0,0,0,0,25,0,0,0,0,0
LAZIENCE,0,0,0,0,0,0,0,75,0,0,0,0,0,0,0,0,0,0,0,25,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
WLACZ,0,0,0,0,0,0,0,0,50,0,0,0,0,0,0,0,0,0,25,0,0,0,0,0,0,0,0,0,0,0,0,25,0,0,0,0
RADIO,0,0,0,0,0,0,0,0,0,50,0,0,0,0,25,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,25,0,0,0,0,0


In [35]:
success_rate = (172-error)/172*100
print('Success rate: %.2f percent'%(success_rate))

Success rate: 52.91 percent
